In [1]:
# Core
import numpy as np
import pandas as pd

# Tensorflow / ML
import tensorflow as tf
from tensorflow.python.keras import backend as K

# Local
from ztf_data import load_ztf_nearest_ast, calc_hit_freq, load_ztf_batch,make_ztf_batch
from asteroid_integrate import load_ast_elt, load_ast_pos
from candidate_element import orbital_element_batch, perturb_elts, random_elts

from asteroid_model import make_model_ast_pos, make_model_ast_dir
from asteroid_model import AsteroidPosition, AsteroidDirection

Found 4 GPUs.  Setting memory growth = True.


## Load ZTF Data and Batch of Orbital Elements

In [2]:
# Load orbital elements for known asteroids
ast_elt = load_ast_elt()

# Number of asteroids
N_ast = ast_elt.shape[0]

In [3]:
# Load ztf nearest asteroid data
ztf_ast = load_ztf_nearest_ast()

In [4]:
# Asteroid numbers and hit counts
ast_num, hit_count = calc_hit_freq(ztf=ztf_ast, thresh_sec=2.0)

# Sort the hit counts in descending order and find the top batch_size
idx = np.argsort(hit_count)[::-1]

# Extract the asteroid number and hit count for this batch
ast_num_best = ast_num[idx]
hit_count_best = hit_count[idx]

In [5]:
# Set batch size
batch_size = 64

# Batch of unperturbed elements
elts_ast = orbital_element_batch(ast_nums=ast_num_best[0:batch_size])

In [6]:
elts_ast

,element_id,a,e,inc,Omega,omega,f,epoch
0,51921,2.669306,0.217361,0.499554,4.699703,2.450796,-1.133491,58600.0
1,59244,2.634727,0.262503,0.465045,5.738297,1.766995,-1.601363,58600.0
2,15786,1.883227,0.047655,0.392360,6.134689,0.804823,-1.246069,58600.0
3,3904,2.556387,0.098279,0.261542,5.450163,2.202423,-1.357345,58600.0
4,142999,2.619944,0.191376,0.514017,0.238022,0.946463,-1.299301,58600.0
...,...,...,...,...,...,...,...,...
59,11952,2.219650,0.086091,0.117967,0.042442,2.904823,-3.016580,58600.0
60,134815,2.612770,0.140831,0.513923,0.272689,0.645552,-0.957836,58600.0
61,27860,2.619406,0.096185,0.200633,5.541399,3.266046,3.948770,58600.0
62,85937,2.342292,0.197267,0.439063,5.279693,3.210025,3.947687,58600.0


In [7]:
# Perturb orbital elements
sigma_a = 0.0 
sigma_e = 0.0 
sigma_f_deg = 0.1
sigma_Omega_deg = 0.0
sigma_omega_deg = 0.0
mask_pert = None
random_seed = 42

elts_pert = perturb_elts(elts_ast, sigma_a=sigma_a, sigma_e=sigma_e, sigma_f_deg=sigma_f_deg, 
                         sigma_Omega_deg=sigma_Omega_deg, sigma_omega_deg=sigma_omega_deg,
                         mask_pert=mask_pert, random_seed=random_seed)

In [8]:
elts_pert

,element_id,a,e,inc,Omega,omega,f,epoch
0,51921,2.669306,0.217361,0.499554,4.699703,2.450796,-1.133117,58600.0
1,59244,2.634727,0.262503,0.465045,5.738297,1.766995,-1.603537,58600.0
2,15786,1.883227,0.047655,0.392360,6.134689,0.804823,-1.245767,58600.0
3,3904,2.556387,0.098279,0.261542,5.450163,2.202423,-1.356673,58600.0
4,142999,2.619945,0.191376,0.514017,0.238022,0.946463,-1.300844,58600.0
...,...,...,...,...,...,...,...,...
59,11952,2.219650,0.086091,0.117967,0.042442,2.904823,-3.014978,58600.0
60,134815,2.612770,0.140831,0.513923,0.272689,0.645552,-0.954132,58600.0
61,27860,2.619406,0.096185,0.200633,5.541399,3.266046,3.950572,58600.0
62,85937,2.342292,0.197267,0.439063,5.279693,3.210025,3.945035,58600.0


## Batches of ZTF Data vs. Elements

In [9]:
# Arguments to make_ztf_batch
thresh_deg = 1.0
near_ast = False
regenerate = False

In [10]:
# Load unperturbed element batch
ztf_elt_ast = load_ztf_batch(elts=elts_ast, thresh_deg=thresh_deg, near_ast=near_ast, regenerate=regenerate)

In [11]:
# Load perturbed element batch
ztf_elt_pert = load_ztf_batch(elts=elts_pert, thresh_deg=thresh_deg, near_ast=near_ast, regenerate=regenerate)

In [12]:
ztf_elt_ast

,ztf_id,element_id,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,...,vz,elt_ux,elt_uy,elt_uz,elt_r,s,s_sec,z,v,is_hit
0,53851,733,b'ZTF18abnothj',594197584815010004,5501,58348.197581,266.229165,-13.513802,-0.063945,-0.983101,...,0.004080,-0.057300,-0.982042,0.179751,2.234078,0.010624,2191.371398,0.999944,0.370539,False
1,73604,733,b'ZTF18ablwzmb',594197584815015003,5501,58348.197581,265.761024,-13.509148,-0.071871,-0.982578,...,0.004080,-0.057300,-0.982042,0.179751,2.234078,0.016809,3467.103003,0.999859,0.927533,False
2,82343,733,b'ZTF18abiydvm',635193253015015018,12089,58389.193252,270.331454,-11.244934,0.005674,-0.977422,...,0.003825,0.000918,-0.977996,0.208622,2.703478,0.005450,1124.142942,0.999985,0.097510,False
3,257221,733,b'ZTF18acakcqg',931471223715015007,39920,58685.471227,29.693832,42.180412,0.643725,0.603886,...,-0.001953,0.639004,0.610779,0.467571,2.175851,0.008712,1797.042210,0.999962,0.249184,False
4,327000,733,b'ZTF18achmdmw',937465970615015011,40837,58691.465972,33.104905,44.059131,0.601970,0.636719,...,-0.002129,0.606278,0.637608,0.475272,2.114865,0.007949,1639.537152,0.999968,0.207418,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90205,5650588,324582,b'ZTF20aaqvhld',1150176701515015008,96618,58904.176701,44.164238,29.650540,0.623416,0.752309,...,-0.001541,0.627640,0.750696,0.206212,2.981799,0.008187,1688.636853,0.999966,0.220027,False
90206,5650589,324582,b'ZTF20aaqvhld',1150176245715015005,96617,58904.176250,44.164062,29.650536,0.623417,0.752307,...,-0.001541,0.627641,0.750695,0.206213,2.981793,0.008187,1688.600639,0.999966,0.220018,False
90207,5650665,324582,b'ZTF20aaqvhll',1150176245815015010,96617,58904.176250,44.368640,28.490480,0.628284,0.753618,...,-0.001541,0.627641,0.750695,0.206213,2.981793,0.013370,2757.856469,0.999911,0.586871,False
90208,5650697,324582,b'ZTF20aaqvhmb',1150176246015015005,96617,58904.176250,43.296207,29.505908,0.633424,0.743491,...,-0.001541,0.627641,0.750695,0.206213,2.981793,0.012388,2555.279465,0.999923,0.503822,False


In [13]:
ztf_elt_ast.columns

Index(['ztf_id', 'element_id', 'ObjectID', 'CandidateID', 'TimeStampID', 'mjd',
       'ra', 'dec', 'ux', 'uy', 'uz', 'qx', 'qy', 'qz', 'vx', 'vy', 'vz',
       'elt_ux', 'elt_uy', 'elt_uz', 'elt_r', 's', 's_sec', 'z', 'v',
       'is_hit'],
      dtype='object')

In [14]:
# Review results
ztf_elt = ztf_elt_ast
element_id_best = ast_num_best[0]
mask = (ztf_elt.element_id == element_id_best)
hits_best = np.sum(ztf_elt[mask].is_hit)
s_sec_min = np.min(ztf_elt[mask].s_sec)
idx = np.argmin(ztf_elt.s)
ztf_id = ztf_elt.ztf_id[idx]
# ztf_elt[mask].iloc[idx:idx+1]
print(f'Best asteroid has element_id = {element_id_best}')
print(f'Hit count: {hits_best}')
print(f'Closest hit: {s_sec_min:0.3f} arc seconds')
# ztf_elt[mask]

Best asteroid has element_id = 51921
Hit count: 158
Closest hit: 0.382 arc seconds


## Load Position and Direction Models

In [15]:
# Data types
dtype = tf.float32
dtype_np = np.float32

In [16]:
# Alias ztf_elt
ztf_elt = ztf_elt_ast.copy()

# Get observation count per element
row_lengths = ztf_elt.element_id.groupby(ztf_elt.element_id).count().values.astype(np.int32)

In [17]:
# Build ragged tensor of input times
ts = tf.RaggedTensor.from_row_lengths(values=ztf_elt.mjd.values.astype(dtype_np), row_lengths=row_lengths)

# Observation times as flattened array
ts_flat = ts.values

In [18]:
# Unique times
ts_unq = np.unique(ztf_elt_ast.mjd)
TimeStampID_unq = np.unique(ztf_elt_ast.TimeStampID)

# The epoch
epoch0 = elts_ast.epoch[0]

In [19]:
# Report time tensor shapes
print(f'ts.shape={ts.shape}')
print(f'ts_flat.shape={ts_flat.shape}')
print(f'ts_unq.shape={ts_unq.shape}')

ts.shape=(64, None)
ts_flat.shape=(90210,)
ts_unq.shape=(6383,)


In [20]:
# Observation site
site_name = 'palomar'

In [21]:
# Build position model
model_pos = make_model_ast_pos(ts=ts, batch_size=batch_size)

ValueError: in converted code:

    /home/michael/IACS/kepler-sieve/src/asteroid_model.py:311 call  *
        f_t = MeanToTrueAnomaly(name='mean_to_true_anomaly')([M_t, e_t])
    /home/michael/anaconda3/envs/kepler/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:773 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    /home/michael/IACS/kepler-sieve/src/orbital_element.py:543 call  *
        E = MeanToEccentricAnomaly(name='ecc_anomaly')((M, e))
    /home/michael/anaconda3/envs/kepler/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:773 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    /home/michael/IACS/kepler-sieve/src/orbital_element.py:514 call  *
        E, F = MeanToEccentricAnomalyIteration(shape=shape, name='M2E_it_1')(M, e, E, F)
    /home/michael/anaconda3/envs/kepler/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:773 __call__
        outputs = call_fn(

In [ ]:
# # Build direction model
# model_dir = make_model_ast_dir(ts=ts_np, batch_size=batch_size, site_name=site_name)

In [ ]:
# Stack elements as a dict of numpy arrays for prediction
cols_elt = ['a', 'e', 'inc', 'Omega', 'omega', 'f', 'epoch']
elts_ast_dict = {col : elts_ast[col].values for col in cols_elt}
# elts_ast_dict

In [ ]:
# # Predict position model
# q_pred, v_pred = model_pos.predict(elts_ast_dict)

In [ ]:
# # Predict direction model
# u_pred, r_pred = model_dir.predict(elts_ast_dict)

# # Review shape of predictions
# u_pred.shape

## Assemble Tensors for Prototype Model

In [ ]:
# Orbital elements
a = tf.Variable(initial_value=elts_ast.a, dtype=dtype)
e = tf.Variable(initial_value=elts_ast.e, dtype=dtype)
inc = tf.Variable(initial_value=elts_ast.inc, dtype=dtype)
Omega = tf.Variable(initial_value=elts_ast.Omega, dtype=dtype)
omega = tf.Variable(initial_value=elts_ast.omega, dtype=dtype)
f = tf.Variable(initial_value=elts_ast.f, dtype=dtype)
epoch = tf.constant(value=elts_ast.epoch, dtype=dtype)

In [ ]:
# Review ztf_elt on best candidate
mask_best = (ztf_elt.element_id == element_id_best)
ztf_elt_best = ztf_elt[mask_best]
ztf_elt_best

In [ ]:
# Build ragged tensor of u_obs
cols_u_obs = ['ux', 'uy', 'uz']
u_obs_r = tf.RaggedTensor.from_row_lengths(values=ztf_elt[cols_u_obs].values, row_lengths=row_lengths)

# Review shape
print(f'u_obs_r.shape = {u_obs_r.shape}')

In [ ]:
# old version of make_ragged_tensors - does not work on this frame b/c 
# t_np = ztf_elt.mjd.values
# u_np = ztf_elt[cols_u_obs].values
# element_id_np = ztf_elt.element_id.values
# time_batch_size = None

# ts_r, u_r, element_id_r = make_ragged_tensors(t_np=t_np, u_np=u_np, element_id_np=element_id_np, batch_size=batch_size)

In [ ]:
# Latest time
t_max = np.max(ztf_elt.mjd.values)

# Time for padded values
t_pad = t_max + 1.0

In [ ]:
# Convert ragged to 2D
ts_2d = ts.to_tensor(default_value=t_pad)
ts_2d_np = ts_2d.numpy()

print(f'ts_2d.shape = {ts_2d.shape}')

## Calculate Position and Direction Using Layers in asteroid_model.py

In [ ]:
# Alias batch_size to elt_batch_size; disambiguate from time_batch_size
elt_batch_size = batch_size

In [ ]:
# Build position layer
ast_pos_layer = AsteroidPosition(ts, batch_size, name='ast_pos_layer')

In [ ]:
# Build direction layer
direction_layer = AsteroidDirection(ts=ts, site_name=site_name, batch_size=elt_batch_size, name='u_pred')

In [ ]:
# Predict position
q_pred, v_pred = ast_pos_layer(a, e, inc, Omega, omega, f, epoch)

In [ ]:
# Review output
print(f'q_pred.shape = {q_pred.shape}')
print(f'v_pred.shape = {q_pred.shape}')

In [ ]:
# Calibration arrays (flat)
cols_q_ast = ['qx', 'qy', 'qz']
cols_v_ast = ['vx', 'vy', 'vz']
q_ast = ztf_elt[cols_q_ast].values.astype(dtype_np)
v_ast = ztf_elt[cols_v_ast].values.astype(dtype_np)

In [ ]:
# Run calibration
ast_pos_layer.calibrate(elts=elts_ast, q_ast=q_ast, v_ast=v_ast)

In [ ]:
# Check that corrections during calibration aren't too large
mean_dq = np.mean(tf.linalg.norm(tf.reshape(ast_pos_layer.dq.values, (-1,3)), axis=1))
mean_dv = np.mean(tf.linalg.norm(tf.reshape(ast_pos_layer.dv.values, (-1,3)), axis=1))

print(f'Mean calibration adjustments:')
print(f'mean_dq = {mean_dq:6.2e}')
print(f'mean_dq = {mean_dv:6.2e}')

In [ ]:
# Predict direction
u_pred, r_pred = direction_layer(a, e, inc, Omega, omega, f, epoch)

In [ ]:
# Review output
print(f'u_pred.shape = {u_pred.shape}')
print(f'r_pred.shape = {r_pred.shape}')

## Imports for Step by Step Calculations

In [ ]:
keras = tf.keras

In [ ]:
import astropy
from astropy.units import au, day, year

# Local imports
from orbital_element import MeanToTrueAnomaly, TrueToMeanAnomaly
from asteroid_data import get_earth_pos, get_sun_pos_vel
from asteroid_model import ElementToPosition
from ra_dec import calc_topos

In [ ]:
# Constants

# The gravitational constant in ('day', 'AU', 'Msun') coordinates
# sim = rebound.Simulation()
# sim.units = ('day', 'AU', 'Msun')
# G_ = sim.G
# Hard code G
G_ = 0.00029591220828559104
# The gravitational field strength mu = G * (m0 + m1)
# For massless asteroids orbiting the sun with units Msun, m0=1.0, m1=0.0, and mu = G
mu = tf.constant(G_)

# Speed of light; express this in AU / day
light_speed_au_day = astropy.constants.c.to(au / day).value

# Number of spatial dimensions
space_dims = 3

# Data types
dtype = tf.float32
dtype_np = np.float32

In [ ]:
# Build direction layer
direction_layer = AsteroidDirection(ts=ts, site_name=site_name, batch_size=elt_batch_size, name='u_pred')

In [ ]:
# Predict direction layer
u_pred, r_pred = direction_layer(a, e, inc, Omega, omega, f, epoch)

In [ ]:
u_pred.shape

## Step by Step Calculation of XXX